An initial attempt at NWB conversion for NeuraLynx data following "manual" conversion described in https://pynwb.readthedocs.io/en/stable/tutorials/domain/ecephys.html .  Unlike the example(s) there I (Yarik) was trying to identify levels of data and metadata to consider, and also to store them across multiple .nwb files. 

In [ ]:
import numpy as np
import pynwb
from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import NWBFile

import neo

In [ ]:
session_data = '../data/BiconditionalOdor/M040-2020-04-28-CDOD11'
# session_data = '/Users/jimmiegmaz/Desktop/M040-2020-04-28-CDOD11' #for testing

In [ ]:
# Common lab wide metadata
lab_metadata = dict(
    lab="MVDMLab",
    institution="Dartmouth College",
    keywords=["DANDI Pilot"], # arbitrary, so let's promote!
)
# Experiment specific one
experiment_metadata = dict(
    experimenter="Jimmie Gmaz <jim.gmaz@gmail.com>",  # Let's see if nwb can swallow such a record ;)
    experiment_description="Contextual odor discrimination task",
)


In [ ]:
# create a reader
reader = neo.io.NeuralynxIO(dirname=session_data) # TODO: newer version should support: , keep_original_times=True)
reader.parse_header()
print(reader)

In [ ]:
seg = reader.read_segment()

In [ ]:
import os.path as op
import re
filename_metadata = re.match(
    '(?P<subject_id>[A-Za-z0-9]*)-(?P<date>20..-..-..)-(?P<task>[A-Za-z]*)(?P<day_of_recording>[0-9]*)$',
    op.basename(session_data)).groupdict()
assert filename_metadata
filename_metadata

In [ ]:
import time

# Those time stamps are in sub-second and not the one we would want to the "session time"
# time.gmtime(reader.get_event_timestamps()[0][0])
# TODO: figure out where in this 
# TODO: figure out what those timestamps in.

In [ ]:
# Scans through Experimental Keys to extract relevant metadata for NWB file

# name of ExpKeys file
keys_name = session_data + '/'  + filename_metadata['subject_id'] + '_' + filename_metadata['date'].replace('-','_') + '_keys.m'

# read session ExpKeys
with open (keys_name, 'rt') as keys_file:
    exp_keys = keys_file.read()

# list of metadata to extract
metadata_list = ['ExpKeys.species','ExpKeys.hemisphere','ExpKeys.weight','ExpKeys.probeDepth','ExpKeys.target']

# initialize metadata dictionary
metadata_keys = dict.fromkeys(metadata_list)

# extract metadata
for item in exp_keys.split("\n"):
    for field in metadata_list:
        if field in item:
            metadata_keys[field] = re.search('(?<=\=)(.*?)(?=\;)', item).group(0).strip() 
            metadata_keys[field] = re.sub('[^A-Za-z0-9]+', '', metadata_keys[field])
            print(metadata_keys[field])
            
# TODO: add surgery details to ExpKeys, including AP and ML coordinates, change probeDepth to mm,
# add filtering, individual tetrode depth, tetrode referencing

In [ ]:
# Metadata which is likely to come from data files and "promotion" metadata records

# Most likely many could be parsed from the filenames which are likely to encode some of it
# So "heuristical" converter could establish metadata harvesting from the filenames

#
# Session specific
session_metadata = dict(
    session_id="%(subject_id)s-%(date)s" % filename_metadata,
    session_description="Extracellular ephys recording in the left hemisphere of the nucleus accumbens",  # args[0] in nwbfile
    session_start_time=datetime.now(tzlocal()), # TEMP  # args[2] in nwbfile; TODO needs to be datetime
)
subject_metadata = dict(
    subject_id=filename_metadata['subject_id'],
    weight=metadata_keys['ExpKeys.weight'],
    age="TODO",  # duplicate with session_start_time and date_of_birth but why not?
    species=metadata_keys['ExpKeys.species'],
    sex="female",
#     hemisphere=metadata_keys['ExpKeys.hemisphere'],
#     depth=metadata_keys['ExpKeys.probeDepth'],
#     region=metadata_keys['ExpKeys.target'],
    date_of_birth=datetime.now(tzlocal()), # TEMP: TODO
)
surgery_metadata = dict(
    surgery="Headbar on xx/xx/2020, craniotomy over right hemisphere on xx/xx/2020, craniotomy over left hemisphere on xx/xx/2020. All surgeries performed by JG."
)
# Actually probably only "identifier" should be file specific, the rest common across files
# we would like to produce: separate for .ncs, .ntt, behavioral metadata, etc
file_metadata = dict(
    source_script="somescript-not-clear-whyneeds to be not empty if file_name is provided",
    source_script_file_name="TODO", # __file__,
)

# common filename prefix - let's mimic DANDI filenaming convention right away
filename_prefix = "sub-{subject_id}_ses-{session_id}".format(**subject_metadata, **session_metadata)
# the rest will be specific to the corresponding file. E.g. we will have separate
#  - `_probe-<name>_ecephys.nwb` (from each .ncs) - contineous data from each tetrode. probably chunked and compressed
#  - `_???_ecephys.nwb` (from each .ntt) - spike detected windowed data. 
#  - `_behav.mpg` + `_behav.nwb` - video recording and metadata (including those .png?) for behavior component within experiment recording session
# Pretty much we need to establish a framework where EVERY file present would be
# provided

In [ ]:
subject_metadata

Code below would need to follow common pattern 
- create a new NWBFile with common metadata,
- populate with relevant data and metadata
- save

In [ ]:
# Such NWBFile will be created for each separate file, and then fill up with the corresponding
#
filename_suffix = "TODO"
nwbfile = NWBFile(
    identifier="{}_{}".format(filename_prefix, filename_suffix), # args[1] in nwbfile, may be just UUID? not sure why user has to provide it really
    subject=pynwb.file.Subject(**subject_metadata),
    **lab_metadata,
    **experiment_metadata,
    **session_metadata,
    **surgery_metadata,
    **file_metadata,
)

In [ ]:
print(nwbfile.identifier)

In [ ]:
# add electrode metadata
# create probe device
device = nwbfile.create_device(name='silicon probe', description='A4x2-tet-5mm-150-200-121', manufacturer='NeuroNexus')

# for each channel on the probe
for chl in reader.header['unit_channels']:
    
    # get tetrode id
    tetrode = re.search('(?<=TT)(.*?)(?=#)', chl[0]).group(0)
    electrode_name = 'tetrode' + tetrode
    
    # get channel id
    channel = re.search('(?<=#)(.*?)(?=#)', chl[0]).group(0)
           
    if electrode_name not in nwbfile.electrode_groups: # make tetrode if does not exist
    
        description = electrode_name
        location = metadata_keys['ExpKeys.hemisphere'] + ' ' + metadata_keys['ExpKeys.target'] + ' ' + \
            '(' + metadata_keys['ExpKeys.probeDepth'] + ' um)'

        electrode_group = nwbfile.create_electrode_group(electrode_name,
                                                         description=description,
                                                         location=location,
                                                         device=device)
        
    # add channel to tetrode
    nwbfile.add_electrode(id=int(channel),
                          x=-1.2, y=float(metadata_keys['ExpKeys.probeDepth']), z=-1.5,
                          location=metadata_keys['ExpKeys.target'], filtering='none',
                          imp = 0.0, group=nwbfile.electrode_groups[electrode_name])

In [ ]:
#add data
from pynwb.ecephys import ElectricalSeries
from pynwb.ecephys import SpikeEventSeries
from pynwb.ecephys import EventWaveform

# add .ntt files
ephys_waveform = EventWaveform()

# loop through .ntt files
for i, chl in enumerate(reader.header['unit_channels']):
    
    # get tetrode id
    tetrode = re.search('(?<=TT)(.*?)(?=#)', chl[0]).group(0)
    tetrode_name = 'TT' + tetrode
           
    if tetrode_name not in ephys_waveform.spike_event_series: # make tetrode if does not exist
        
        chl_list = []
        
        for j, group in enumerate(nwbfile.electrodes['group']):
        
            if tetrode in nwbfile.electrodes['group'][j].fields['description']:
                
                chl_list.append(j)
        
        electrode_table_region = nwbfile.create_electrode_table_region(chl_list, tetrode_name)
    
        waveform = reader.get_spike_raw_waveforms(unit_index=i)

        ephys_waveform.create_spike_event_series(tetrode_name,
                                                 waveform,
                                                 seg.spiketrains[i].times,
                                                 electrode_table_region)

nwbfile.add_acquisition(ephys_waveform)

# add .ncs files

chl_list = []

for chl in reader.header['signal_channels']['id']:
    
    chl_list.append(nwbfile.electrodes['id'][:].index(chl))
    
electrode_table_region = nwbfile.create_electrode_table_region(chl_list, 'CSC order for time series')

ephys_ts = ElectricalSeries('CSC data',
                            seg.analogsignals[0].magnitude,
                            electrode_table_region,
                            timestamps=seg.analogsignals[0].times,
                            comments='n/a',
                            description='unfiltered CSC data')

nwbfile.add_acquisition(ephys_ts)

# nwbfile.add_unit(id=1, electrodes=[0])
# nwbfile.add_unit(id=2, electrodes=[0])

In [ ]:
# Save the generated file
from pynwb import NWBHDF5IO

# TODO: I think we should right away use dandi-cli provided API to create the filename based on metadata
# in the NWBFile
with NWBHDF5IO('BCD_example.nwb', 'w') as io:
    io.write(nwbfile)